<div style="background-color: lightblue; padding: 10px; border-radius: 5px; font-family: Aptos; ">
    <h1><b> Cultural Tourism Route Optimization </b></h1>


<ul>
    <li><b>Authored by:</b> Uvini Wijesinghe</li>
    <li><b>Duration:</b> 10 Weeks</li>
    <li><b>Level:</b> Intermediate</li>
    <li><b>Pre-requisite Skills:</b> Python</li>
</ul>

</div>
</div>

<div style="font-family: Aptos; font-size: 16px;">
Creating optimized cultural tourism routes in Melbourne involves integrating data from multiple sources, including public memorials, sculptures, artworks, fountains, monuments, and landmarks, along with key transport infrastructure such as City Circle tram stops and Melbourne Visitor Shuttle bus stops. By analyzing pedestrian movement patterns, the objective is to design routes that maximize visitor engagement by guiding them through high-interest cultural sites while ensuring accessibility and efficiency.
</div>

In [167]:
import pandas as pd

### Artworks, Fountains and Monuments

In [169]:
df = pd.read_csv("Datasets/public-artworks-fountains-and-monuments.csv")
df.head(2)

,Asset Type,Name,Xorg,Xsource,Address Point,Artist,Alternate Name,Art Date,Mel way Ref,Respective Author,Structure,Co-ordinates,Easting,Northing
0,Art,Port Phillip Monument,City of Melbourne,MCC - Ortho Image March 2005 - Final,"178 Sims Street, WEST MELBOURNE",unknown,NaN,1941,2S_K11,City Of Melbourne,Basalt monument,"-37.8056957854241, 144.907291041632",315771.745,5813680.208
1,Art,Bird Panels,City of Melbourne,MCC - Ortho Image March 2005 - Final,76 Canning Street,Di Christensen and Bernice McPherson,NaN,1995,2A_E5,City Of Melbourne,Stainless-steel panels,"-37.7953526839703, 144.940687314302",318686.757,5814893.278


<div style="background-color: #eeebeb; padding: 10px; border-radius: 5px; font-family: Aptos; ">
    <h3><b> Train Routes </b></h3>
</div>

In [171]:
metro_train_routes = pd.read_csv("Datasets/gtfs/Metro Train/routes.txt", delimiter=",") 

# Split based on 'aus:vic:vic-' and take the second part
metro_train_routes['train_id'] = metro_train_routes['route_id'].str.extract(r'aus:vic:vic-(.*?):?$', expand=False)

metro_train_routes = metro_train_routes[['train_id', 'route_short_name', 'route_long_name']]

metro_train_routes = metro_train_routes.drop_duplicates()

metro_train_routes.head()

,train_id,route_short_name,route_long_name
0,02-ALM,Alamein,Alamein - City
1,02-BEG,Belgrave,Belgrave - City
2,02-CBE,Cranbourne,Cranbourne - City
3,02-CCL,City Circle,NaN
4,02-CGB,Craigieburn,Craigieburn - City


### Train Stops

In [173]:
metro_train_stops = pd.read_csv("Datasets/gtfs/Metro Train/stops.txt", delimiter=",")

metro_train_stops = metro_train_stops[['stop_id', 'stop_name', 'stop_lat','stop_lon']]

metro_train_stops = metro_train_stops.drop_duplicates()

metro_train_stops['stop_id'] = metro_train_stops['stop_id'].astype(str).str.strip()

metro_train_stops.head()

,stop_id,stop_name,stop_lat,stop_lon
0,10117,Jordanville Station,-37.873763,145.112473
1,10920,Flagstaff Station,-37.811880,144.956043
2,10921,Flagstaff Station,-37.811725,144.955968
3,10922,Melbourne Central Station,-37.809974,144.962547
4,10923,Melbourne Central Station,-37.809865,144.962516


### Train Times

In [175]:
metro_train_times = pd.read_csv("Datasets/gtfs/Metro Train/stop_times.txt", delimiter=",")

metro_train_times['train_id'] = metro_train_times['trip_id'].str.extract(r'(^[^-]+-[^-]+)')

metro_train_times = metro_train_times[['trip_id', 'train_id', 'stop_id', 'stop_sequence']]

metro_train_times = metro_train_times.drop_duplicates()

metro_train_times['stop_id'] = metro_train_times['stop_id'].astype(str).str.strip()

metro_train_times.head()

C:\Users\uvini\AppData\Local\Temp\ipykernel_50288\2317826863.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  metro_train_times = pd.read_csv("Datasets/gtfs/Metro Train/stop_times.txt", delimiter=",")


,trip_id,train_id,stop_id,stop_sequence
0,02-ALM--16-T2-2302,02-ALM,11197,1
1,02-ALM--16-T2-2302,02-ALM,11198,2
2,02-ALM--16-T2-2302,02-ALM,11200,3
3,02-ALM--16-T2-2302,02-ALM,11202,4
4,02-ALM--16-T2-2302,02-ALM,11203,5


In [176]:
unique_train_ids = metro_train_times['train_id'].unique()

stop_count = metro_train_times.groupby(['trip_id', 'train_id'])['stop_sequence'].count().reset_index()
stop_count.rename(columns={'stop_sequence': 'sequence_count'}, inplace=True)
print(stop_count)

                    trip_id train_id  sequence_count
0         02-ALM--1-T2-2302   02-ALM               7
1         02-ALM--1-T2-2304   02-ALM               7
2         02-ALM--1-T2-2305   02-ALM               7
3         02-ALM--1-T2-2306   02-ALM               7
4         02-ALM--1-T2-2307   02-ALM               7
...                     ...      ...             ...
41810  02-WIL--8-T5_z0-X325   02-WIL               4
41811  02-WIL--8-T5_z0-X326   02-WIL               4
41812  02-WIL--8-T5_z0-X327   02-WIL               4
41813  02-WIL--8-T5_z0-X328   02-WIL               4
41814  02-WIL--8-T5_z0-X330   02-WIL               4

[41815 rows x 3 columns]


In [177]:
# Find the highest stop_sequence for each train_id
highest_seq_per_train = metro_train_times.loc[
    metro_train_times.groupby('train_id')['stop_sequence'].idxmax(),
    ['train_id', 'trip_id', 'stop_sequence']
].rename(columns={'stop_sequence': 'max_sequence'})

# Get unique trip_ids
unique_trip_ids = highest_seq_per_train['trip_id'].unique()

# Filter metro_train_times for those trip_ids
filtered_metro_train_times = metro_train_times[metro_train_times['trip_id'].isin(unique_trip_ids)]
filtered_metro_train_times

,trip_id,train_id,stop_id,stop_sequence
2539,02-ALM--16-T5-2801,02-ALM,11213,1
2540,02-ALM--16-T5-2801,02-ALM,22189,2
2541,02-ALM--16-T5-2801,02-ALM,12196,3
2542,02-ALM--16-T5-2801,02-ALM,12198,4
2543,02-ALM--16-T5-2801,02-ALM,12200,5
...,...,...,...,...
669246,02-WIL--16-T2-6300,02-WIL,26508,8
669247,02-WIL--16-T2-6300,02-WIL,15522,9
669248,02-WIL--16-T2-6300,02-WIL,14328,10
669249,02-WIL--16-T2-6300,02-WIL,22192,11


In [178]:
# Trim spaces and convert stop_id to string for consistency
filtered_metro_train_times['stop_id'] = filtered_metro_train_times['stop_id'].astype(str).str.strip()
metro_train_stops['stop_id'] = metro_train_stops['stop_id'].astype(str).str.strip()

result = filtered_metro_train_times.merge(metro_train_stops, on='stop_id', how='left')
result.head(2)

C:\Users\uvini\AppData\Local\Temp\ipykernel_50288\794004814.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_metro_train_times['stop_id'] = filtered_metro_train_times['stop_id'].astype(str).str.strip()


,trip_id,train_id,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
0,02-ALM--16-T5-2801,02-ALM,11213,1,Flinders Street Station,-37.818307,144.966010
1,02-ALM--16-T5-2801,02-ALM,22189,2,Southern Cross Station,-37.818535,144.952144


# Buses

In [81]:
fil = metro_train_times.loc[metro_train_times['trip_id'] == '02-BEG--16-T6-3602']
fil.head(35)

,trip_id,train_id,stop_id,stop_sequence
26136,02-BEG--16-T6-3602,02-BEG,11120,1
26137,02-BEG--16-T6-3602,02-BEG,11121,2
26138,02-BEG--16-T6-3602,02-BEG,11122,3
26139,02-BEG--16-T6-3602,02-BEG,11245,4
26140,02-BEG--16-T6-3602,02-BEG,11247,5
26141,02-BEG--16-T6-3602,02-BEG,11249,6
26142,02-BEG--16-T6-3602,02-BEG,11410,7
26143,02-BEG--16-T6-3602,02-BEG,11412,8
26144,02-BEG--16-T6-3602,02-BEG,12237,9
26145,02-BEG--16-T6-3602,02-BEG,12234,10


In [83]:
fil.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31 entries, 26136 to 26166
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   trip_id        31 non-null     object
 1   train_id       31 non-null     object
 2   stop_id        31 non-null     object
 3   stop_sequence  31 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.2+ KB


In [85]:
# fil = metro_train_stops.loc[metro_train_stops['stop_id'] == '11120']
# fil.head(35)

In [89]:
# Trim spaces and convert stop_id to string for consistency
fil['stop_id'] = fil['stop_id'].astype(str).str.strip()
metro_train_stops['stop_id'] = metro_train_stops['stop_id'].astype(str).str.strip()

result = fil.merge(metro_train_stops, on='stop_id', how='left')
result.head(35)

C:\Users\uvini\AppData\Local\Temp\ipykernel_50288\3998045313.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fil['stop_id'] = fil['stop_id'].astype(str).str.strip()


,trip_id,train_id,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
0,02-BEG--16-T6-3602,02-BEG,11120,1,Belgrave Station,-37.909100,145.355132
1,02-BEG--16-T6-3602,02-BEG,11121,2,Tecoma Station,-37.908090,145.342904
2,02-BEG--16-T6-3602,02-BEG,11122,3,Upwey Station,-37.903710,145.331413
3,02-BEG--16-T6-3602,02-BEG,11245,4,Upper Ferntree Gully Station,-37.892596,145.307293
4,02-BEG--16-T6-3602,02-BEG,11247,5,Ferntree Gully Station,-37.881342,145.294952
5,02-BEG--16-T6-3602,02-BEG,11249,6,Boronia Station,-37.860529,145.284736
6,02-BEG--16-T6-3602,02-BEG,11410,7,Bayswater Station,-37.841866,145.268179
7,02-BEG--16-T6-3602,02-BEG,11412,8,Heathmont Station,-37.828494,145.244563
8,02-BEG--16-T6-3602,02-BEG,12237,9,Ringwood Station,-37.815924,145.229070
9,02-BEG--16-T6-3602,02-BEG,12234,10,Heatherdale Station,-37.818832,145.214584


In [166]:
metro_train_times['stop_id'] = metro_train_times['stop_id'].astype(str).str.strip()
metro_train_stops['stop_id'] = metro_train_stops['stop_id'].astype(str).str.strip()

result_df = pd.merge(metro_train_times, metro_train_stops, on='stop_id')
result_df.head()

,train_id,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
0,02-ALM,11197,1,Alamein Station,-37.868204,145.079727
1,02-ALM,11198,2,Ashburton Station,-37.861932,145.081390
2,02-ALM,11200,3,Burwood Station,-37.851744,145.080540
3,02-ALM,11202,4,Hartwell Station,-37.843883,145.075426
4,02-ALM,11203,5,Willison Station,-37.835432,145.070055


In [182]:
fil = result_df.loc[result_df['train_id'] == '02-BEG']
fil.head(30)

,train_id,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
73,02-BEG,11120,1,Belgrave Station,-37.909100,145.355132
74,02-BEG,11121,2,Tecoma Station,-37.908090,145.342904
75,02-BEG,11122,3,Upwey Station,-37.903710,145.331413
76,02-BEG,11245,4,Upper Ferntree Gully Station,-37.892596,145.307293
77,02-BEG,11247,5,Ferntree Gully Station,-37.881342,145.294952
78,02-BEG,11249,6,Boronia Station,-37.860529,145.284736
79,02-BEG,11410,7,Bayswater Station,-37.841866,145.268179
80,02-BEG,11412,8,Heathmont Station,-37.828494,145.244563
81,02-BEG,12237,9,Ringwood Station,-37.815924,145.229070
82,02-BEG,12234,10,Heatherdale Station,-37.818832,145.214584


In [184]:
fil.info()

<class 'pandas.core.frame.DataFrame'>
Index: 232 entries, 73 to 304
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   train_id       232 non-null    object 
 1   stop_id        232 non-null    object 
 2   stop_sequence  232 non-null    int64  
 3   stop_name      232 non-null    object 
 4   stop_lat       232 non-null    float64
 5   stop_lon       232 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 12.7+ KB


,train_id,stop_id,stop_sequence
0,02-ALM,11197,1
1,02-ALM,11198,2


In [79]:
result_df = pd.merge(metro_train_times, metro_train_stops, on='stop_id')
result_df.head()

ValueError: You are trying to merge on int64 and object columns for key 'stop_id'. If you wish to proceed you should use pd.concat